# Milestone 3: Model Training and Evaluation with PyTorch Lightning

## Part 1: Benchmarking Feedforward NN vs. RNN on Sequence Data

### A. Dataset: Human Activity Recognition Using Smartphones

Name: [UCI HAR Dataset (Human Activity Recognition)]

Link: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

The Human Activity Recognition database was built from the recordings of 30 subjects, aged 19 to 48 years, performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The activities involved walking, walking upstairs, walking downstairs, sitting, standing, and lying down. The obtained dataset has been randomly partitioned into two sets, where 70% of the volunteers were selected for generating the training data and 30% for the test data. The models predict the activity being performed based on the test data.

### B. Data Preparation
1. Creating a custom dataset class

In [1]:
# %load /Users/km/Desktop/project/datasets.py
# datasets.py
import os
import numpy as np
import torch
from torch.utils.data import Dataset

class HARSequenceDataset(Dataset):
    def __init__(self, data_dir, split='train'):
        signal_dir = os.path.join(data_dir, split, 'Inertial Signals')

        signal_files = [
            'body_acc_x_', 'body_acc_y_', 'body_acc_z_',
            'body_gyro_x_', 'body_gyro_y_', 'body_gyro_z_',
            'total_acc_x_', 'total_acc_y_', 'total_acc_z_'
        ]

        # Load and stack all signals → shape: [num_samples, 128, 9]
        signals = []
        for signal in signal_files:
            path = os.path.join(signal_dir, signal + split + '.txt')
            data = np.loadtxt(path)
            signals.append(data)
        
        X = np.stack(signals, axis=-1)  # Shape: (num_samples, 128, 9)

        # Load labels (1-based → make it 0-based)
        y_path = os.path.join(data_dir, split, 'y_' + split + '.txt')
        y = np.loadtxt(y_path).astype(np.int64) - 1

        # Normalize features (optional but recommended)
        X = (X - X.mean()) / X.std()

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


2. Building DataLoader

In [2]:
# %load /Users/km/Desktop/project/utils/data_loader.py
# data_loader.py

from torch.utils.data import DataLoader
from datasets import HARSequenceDataset

def get_dataloaders(data_dir, batch_size=64):
    train_ds = HARSequenceDataset(data_dir, split='train')
    val_ds = HARSequenceDataset(data_dir, split='val')
    test_ds = HARSequenceDataset(data_dir, split='test')

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


3. Setting up imports

In [3]:
# %load /Users/km/Desktop/project/models/base.py
# base.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy


### C. Model Implementation with PyTorch Lightning
1. Feedforward Neural Network (FFNN):

    Key Points:
    - Flattens sequence input: [batch_size, 128, 9] → [batch_size, 128 * 9]

    - Simple linear layers.

In [4]:
# %load /Users/km/Desktop/project/models/ffnn.py
# ffnn.py

from models.base import *

class FFNNModel(pl.LightningModule):
    def __init__(self, input_dim=128*9, num_classes=6, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

        self.accuracy = Accuracy(task='multiclass', num_classes=num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        return self.model(x)

    def _step(self, batch, stage):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = self.accuracy(logits.softmax(dim=-1), y)
        self.log(f'{stage}_loss', loss, prog_bar=True)
        self.log(f'{stage}_acc', acc, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._step(batch, 'train')

    def validation_step(self, batch, batch_idx):
        self._step(batch, 'val')

    def test_step(self, batch, batch_idx):
        self._step(batch, 'test')

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


2. Recurrent Neural Network (RNN):
    
    Key Points:
    - Uses final hidden state from LSTM for classification.

    - Optionally switch to GRU with a small change.

In [5]:
# %load /Users/km/Desktop/project/models/rnn.py
# rnn.py

from models.base import *

class RNNModel(pl.LightningModule):
    def __init__(self, input_size=9, hidden_size=128, num_layers=2, num_classes=6, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()

        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=0.3)

        self.fc = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        # x: [batch, seq_len, features]
        out, (hn, cn) = self.lstm(x)
        logits = self.fc(hn[-1])  # Use last hidden state
        return logits

    def _step(self, batch, stage):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = self.accuracy(logits.softmax(dim=-1), y)
        self.log(f'{stage}_loss', loss, prog_bar=True)
        self.log(f'{stage}_acc', acc, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._step(batch, 'train')

    def validation_step(self, batch, batch_idx):
        self._step(batch, 'val')

    def test_step(self, batch, batch_idx):
        self._step(batch, 'test')

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


### D. Benchmarking and Evaluation
Training script with TensorBoard

In [6]:
# %load /Users/km/Desktop/project/train.py
# train.py

# train.py

import sys
import os
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from utils.data_loader import get_dataloaders
from models.ffnn import FFNNModel
from models.rnn import RNNModel

def train(model_type='ffnn'):
    log_dir = os.path.join(os.getcwd(), "/Users/km/Desktop/project/logs")
    logger = TensorBoardLogger(log_dir, name=model_type)
    # logger = TensorBoardLogger("logs", name=model_type)
    train_loader, val_loader, test_loader = get_dataloaders("/Users/km/Desktop/project/data/har/UCI HAR Dataset", batch_size=64)

    if model_type == 'ffnn':
        model = FFNNModel()
    elif model_type == 'rnn':
        model = RNNModel()
    else:
        raise ValueError("Unknown model_type")

    trainer = pl.Trainer(
        max_epochs=20,
        logger=logger,
        accelerator='auto',
        devices='auto',
        log_every_n_steps=10
    )

    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, dataloaders=test_loader)

if __name__ == "__main__":
    model_type = sys.argv[1]
    train(model_type)



ValueError: Unknown model_type

1. Training FFNN Model

In [7]:
%run /Users/km/Desktop/project/train.py ffnn

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | model    | Sequential         | 328 K  | train
1 | accuracy | MulticlassAccuracy | 0      | train
2 | loss_fn  | CrossEntropyLoss   | 0      | train
--------------------------------------------------------
328 K     Trainable params
0         Non-trainable params
328 K     Total params
1.315     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8917543292045593     │
│         test_loss         │    0.6541621685028076     │
└───────────────────────────┴───────────────────────────┘

- Training Time = 33.5 seconds
- Test Accuracy = 89.96%
- Test Loss = 60.85%

2. Training RNN Model

In [8]:
%run /Users/km/Desktop/project/train.py rnn

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | lstm     | LSTM               | 203 K  | train
1 | fc       | Linear             | 774    | train
2 | loss_fn  | CrossEntropyLoss   | 0      | train
3 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
204 K     Trainable params
0         Non-trainable params
204 K     Total params
0.816     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9212758541107178     │
│         test_loss         │    0.23137536644935608    │
└───────────────────────────┴───────────────────────────┘

- Training Time = 114.5 seconds
- Test Accuracy = 90.94%
- Test Loss = 28.20%

Launching TensorBoard

In [10]:
%reload_ext tensorboard
%tensorboard --logdir="/Users/km/Desktop/project/logs"

Reusing TensorBoard on port 6006 (pid 22995), started 0:01:18 ago. (Use '!kill 22995' to kill it.)

## Part 2: Benchmarking Feedforward NN vs. CNN on Image Data
### A. Dataset: FashionMNIST (28×28 grayscale images of clothing)
Name: FashionMNIST

Link: https://github.com/zalandoresearch/fashion-mnist

FashionMNIST is a drop-in replacement for the original MNIST dataset. It contains 28x28 grayscale images of 10 fashion categories, such as: T-shirt/top; Trouser; Pullover; Dress; Coat; Sandal; Shirt; Sneaker; Bag; Ankle boot

### B. Data Preparation
Creating a custom dataset class and building a DataLoader

In [11]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Image transforms: Normalize to mean 0.5, std 0.5 (standard for grayscale)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Use a temporary directory that allows writing
data_root = "/tmp/fashionmnist"

# Make sure the directory exists
os.makedirs(data_root, exist_ok=True)

# Download dataset
train_data = datasets.FashionMNIST(root=data_root, train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root=data_root, train=False, download=True, transform=transform)

# Train/val split
train_data, val_data = random_split(train_data, [50000, 10000])

# Dataloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)


100%|██████████| 26.4M/26.4M [00:02<00:00, 9.87MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 263kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 4.49MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 7.59MB/s]


### C. Model Implementation with PyTorch Lightning
1. Feedforward Neural Network (FFNN):

    Key Points:
    - Flattened input
    - 3 layers (784 → 256 → 64 → 10)

In [12]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F

class FFNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss, on_step=False, on_epoch=True)
        self.log("test_acc", acc, on_step=False, on_epoch=True)

    def on_test_epoch_end(self):
        test_loss = self.trainer.callback_metrics.get("test_loss")
        test_acc = self.trainer.callback_metrics.get("test_acc")

        if test_loss is not None:
            self.logger.experiment.add_scalar("test_loss", test_loss, self.current_epoch)
        if test_acc is not None:
            self.logger.experiment.add_scalar("test_acc", test_acc, self.current_epoch)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


2. Convolutional Neural Network (CNN):

    Key Points:
    - 2 Conv+Pool blocks
    - Fully connected after flattening (64×7×7 → 128 → 10)

In [13]:
class CNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),  # 28x28 → 28x28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # 28x28 → 14x14
            nn.Conv2d(32, 64, 3, padding=1), # 14x14 → 14x14
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # 14x14 → 7x7
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.convnet(x)
        return self.classifier(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss, on_step=False, on_epoch=True)
        self.log("test_acc", acc, on_step=False, on_epoch=True)


    def on_test_epoch_end(self):
        test_loss = self.trainer.callback_metrics.get("test_loss")
        test_acc = self.trainer.callback_metrics.get("test_acc")

        if test_loss is not None:
            self.logger.experiment.add_scalar("test_loss", test_loss, self.current_epoch)
        if test_acc is not None:
            self.logger.experiment.add_scalar("test_acc", test_acc, self.current_epoch)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


### D. Benchmarking and Evaluation
Training script with TensorBoard

In [14]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

# Writable log directory
logger = TensorBoardLogger(save_dir="/Users/km/Desktop/project/logs", name="fashion_ffnn")

1. Training FFNN Model

In [15]:
# Train FFNN
ffnn_model = FFNN()
trainer_ffnn = Trainer(max_epochs=5, logger=logger)
trainer_ffnn.fit(ffnn_model, train_loader, val_loader)
trainer_ffnn.test(ffnn_model, dataloaders=test_loader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 218 K  | train
---------------------------------------------
218 K     Trainable params
0         Non-trainable params
218 K     Total params
0.872     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8626999855041504     │
│         test_loss         │    0.3844006657600403     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3844006657600403, 'test_acc': 0.8626999855041504}]

Training CNN Model

In [16]:
# Train CNN
cnn_model = CNN()
trainer_cnn = Trainer(max_epochs=5, logger=logger)
trainer_cnn.fit(cnn_model, train_loader, val_loader)
trainer_cnn.test(cnn_model, dataloaders=test_loader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/km/Desktop/project/logs/fashion_ffnn/version_5/checkpoints exists and is not empty.

  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | convnet    | Sequential | 18.8 K | train
1 | classifier | Sequential | 402 K  | train
--------------------------------------------------
421 K     Trainable params
0         Non-trainable params
421 K     Total params
1.687     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9093999862670898     │
│         test_loss         │    0.24401989579200745    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.24401989579200745, 'test_acc': 0.9093999862670898}]

Launching TensorBoard

In [17]:
%reload_ext tensorboard
%tensorboard --logdir="/Users/km/Desktop/project/logs"

Reusing TensorBoard on port 6006 (pid 22995), started 0:05:15 ago. (Use '!kill 22995' to kill it.)

## Part 3: Comparing Optimizers and Analyzing Training Curves
### A. Experiment Setup
Using the same FFNN Model from Part 2.

1. Optimizer = Adam

In [18]:
import torch
from torch import nn
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# --------------------- Data Module ---------------------
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def prepare_data(self):
        FashionMNIST(root="/tmp", train=True, download=True)
        FashionMNIST(root="/tmp", train=False, download=True)

    def setup(self, stage=None):
        full = FashionMNIST(root="/tmp", train=True, transform=self.transform)
        self.train_set, self.val_set = random_split(full, [50000, 10000])
        self.test_set = FashionMNIST(root="/tmp", train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

# --------------------- FFNN Model ---------------------
class FFNNClassifier(pl.LightningModule):
    def __init__(self, optimizer_type="Adam"):
        super().__init__()
        self.optimizer_type = optimizer_type
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss)
        self.log("test_acc", acc)

    def configure_optimizers(self):
        if self.optimizer_type == "SGD":
            return torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        elif self.optimizer_type == "RMSProp":
            return torch.optim.RMSprop(self.parameters(), lr=1e-3)
        else:  # Default to Adam
            return torch.optim.Adam(self.parameters(), lr=1e-3)

# --------------------- Run Training ---------------------
def train_with_optimizer(opt_name):
    model = FFNNClassifier(optimizer_type=opt_name)
    dm = FashionMNISTDataModule(batch_size=64)
    logger = TensorBoardLogger(save_dir="/Users/km/Desktop/project/logs", name=f"FFNN_{opt_name}")
    checkpoint = ModelCheckpoint(monitor="val_acc", mode="max")
    trainer = pl.Trainer(max_epochs=5, logger=logger, callbacks=[checkpoint], log_every_n_steps=10)
    trainer.fit(model, dm)
    trainer.test(model, dataloaders=dm.test_dataloader())


train_with_optimizer("Adam")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
100%|██████████| 26.4M/26.4M [00:03<00:00, 8.66MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 261kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 4.08MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.42MB/s]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Sequential       | 218 K  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
218 K     Trainable params
0         Non-trainable params
218 K     Total params
0.872     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8611000180244446     │
│         test_loss         │    0.3929591178894043     │
└───────────────────────────┴───────────────────────────┘

2. Optimizer = SGD

In [19]:
import torch
from torch import nn
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# --------------------- Data Module ---------------------
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def prepare_data(self):
        FashionMNIST(root="/tmp", train=True, download=True)
        FashionMNIST(root="/tmp", train=False, download=True)

    def setup(self, stage=None):
        full = FashionMNIST(root="/tmp", train=True, transform=self.transform)
        self.train_set, self.val_set = random_split(full, [50000, 10000])
        self.test_set = FashionMNIST(root="/tmp", train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

# --------------------- FFNN Model ---------------------
class FFNNClassifier(pl.LightningModule):
    def __init__(self, optimizer_type="SGD"):
        super().__init__()
        self.optimizer_type = optimizer_type
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss)
        self.log("test_acc", acc)

    def configure_optimizers(self):
        if self.optimizer_type == "SGD":
            return torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        elif self.optimizer_type == "RMSProp":
            return torch.optim.RMSprop(self.parameters(), lr=1e-3)
        else:  # Default to Adam
            return torch.optim.Adam(self.parameters(), lr=1e-3)

# --------------------- Run Training ---------------------
def train_with_optimizer(opt_name):
    model = FFNNClassifier(optimizer_type=opt_name)
    dm = FashionMNISTDataModule(batch_size=64)
    logger = TensorBoardLogger(save_dir="/Users/km/Desktop/project/logs", name=f"FFNN_{opt_name}")
    checkpoint = ModelCheckpoint(monitor="val_acc", mode="max")
    trainer = pl.Trainer(max_epochs=5, logger=logger, callbacks=[checkpoint], log_every_n_steps=10)
    trainer.fit(model, dm)
    trainer.test(model, dataloaders=dm.test_dataloader())


train_with_optimizer("SGD")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Sequential       | 218 K  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
218 K     Trainable params
0         Non-trainable params
218 K     Total params
0.872     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8604999780654907     │
│         test_loss         │    0.37801334261894226    │
└───────────────────────────┴───────────────────────────┘

3. Optimizer = RMSProp

In [20]:
import torch
from torch import nn
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# --------------------- Data Module ---------------------
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def prepare_data(self):
        FashionMNIST(root="/tmp", train=True, download=True)
        FashionMNIST(root="/tmp", train=False, download=True)

    def setup(self, stage=None):
        full = FashionMNIST(root="/tmp", train=True, transform=self.transform)
        self.train_set, self.val_set = random_split(full, [50000, 10000])
        self.test_set = FashionMNIST(root="/tmp", train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

# --------------------- FFNN Model ---------------------
class FFNNClassifier(pl.LightningModule):
    def __init__(self, optimizer_type="RMSProp"):
        super().__init__()
        self.optimizer_type = optimizer_type
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss)
        self.log("test_acc", acc)

    def configure_optimizers(self):
        if self.optimizer_type == "SGD":
            return torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        elif self.optimizer_type == "RMSProp":
            return torch.optim.RMSprop(self.parameters(), lr=1e-3)
        else:  # Default to Adam
            return torch.optim.Adam(self.parameters(), lr=1e-3)

# --------------------- Run Training ---------------------
def train_with_optimizer(opt_name):
    model = FFNNClassifier(optimizer_type=opt_name)
    dm = FashionMNISTDataModule(batch_size=64)
    logger = TensorBoardLogger(save_dir="/Users/km/Desktop/project/logs", name=f"FFNN_{opt_name}")
    checkpoint = ModelCheckpoint(monitor="val_acc", mode="max")
    trainer = pl.Trainer(max_epochs=5, logger=logger, callbacks=[checkpoint], log_every_n_steps=10)
    trainer.fit(model, dm)
    trainer.test(model, dataloaders=dm.test_dataloader())


train_with_optimizer("RMSProp")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Sequential       | 218 K  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
218 K     Trainable params
0         Non-trainable params
218 K     Total params
0.872     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8539000153541565     │
│         test_loss         │    0.4031919240951538     │
└───────────────────────────┴───────────────────────────┘

Launching TensorBoard

In [21]:
%reload_ext tensorboard
%tensorboard --logdir="/Users/km/Desktop/project/logs"

Reusing TensorBoard on port 6006 (pid 22995), started 0:12:32 ago. (Use '!kill 22995' to kill it.)